## Using API classes to get data and save to db

# <span style="color:red">clear all output before saving: db output contains passwords! </span>
 
- importing modules needed
- creates a temporary db using .env
- creates the API objects for each vendor/station type
- pulls data from the vendor API


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:

from ewxpwsdb.db.models import WeatherStation, APIResponse, Reading, StationType
from ewxpwsdb.db.importdata import import_station_file, read_station_table

In [ ]:
station_file = '../data/test_stations.tsv'
station_type = 'RAINWISE'

### create new temp database to work with

this notebook does not use the .env file or the get_db_url() function for the app's db, 
instead it creates a new file to be used for a temporary, empty postgresql database, deleted in the end. 
this requires postgresql to be installed on your computer and running.  Any version should work. Tested with versions 15 and 16

Note as of now the functions used assume that the pg server can be accessed without a password from localhost. 

On MacOS, https://postgresapp.com is the very fastest way to get started and works in this way. 

In [ ]:
# Postgresql temporary URL

# note, if you run this twice without running the associated drop function,
# there will be an orphaned empty temp database you may have to drop manually because the temp db name will be lost
# see the end of this notebook for that function
from ewxpwsdb.db.database import create_temp_pg_engine, get_db_url
db_url =  get_db_url()  # loads from standard environment, must be a db url with user that has admin privs to create new db
engine = create_temp_pg_engine(admin_db_url = db_url)

temp_db_url = engine.url
temp_db_url.database


In [ ]:

# use the newly created temporary database

from ewxpwsdb.db.database import Session, init_db
from sqlmodel import select
init_db(engine, station_tsv_file=station_file)



## work with db

This works for just one station type.  Set the type you'd like to work with here.  See `src/ewxpwsdb/weather_apis/__init__.py` file for the list of currently supported station types.   

In [ ]:
from ewxpwsdb.weather_apis import STATION_TYPE_LIST, API_CLASS_TYPES
APIclass = API_CLASS_TYPES[station_type]
print(APIclass._sampling_interval)

The following function pulls one station of a particular type, the first one it finds

In [ ]:
# conventience function get the first station from the database for a specific type
def get_one_station(station_type, engine = engine):
    with Session(engine) as session:
        statement = select(WeatherStation).where(WeatherStation.station_type == station_type)
        results = session.exec(statement)
        weather_station = results.first()

    return(weather_station)

### create station object from database

In [ ]:
station = get_one_station(station_type)
station

 *could create weather station object without database, but then there is no ID Field which then can't be used to create related objects*

In [ ]:

# this is example code to pull a station from a file of stations, and not a database

# stations = read_station_table(station_file)
# # add code to find the correct one
# station_data = list(filter(lambda x: (x['station_type']==station_type), stations))[0]
# station = WeatherStation.model_validate(station_data) 
# station.station_type


## Test APIs


In [ ]:
# create API class from station

from ewxpwsdb.weather_apis import API_CLASS_TYPES
wapi = API_CLASS_TYPES[station.station_type](station)
wapi.station_type

In [ ]:
# demo how the weatherapi interface works

print(wapi.station_type)
print(wapi.weather_station.id)
print(wapi.sampling_interval)
print(wapi.APIConfigClass)
# check that configuration class is instantiated with same data in database
api_config = wapi.APIConfigClass.model_validate_json_str(wapi.weather_station.api_config)
print(api_config == wapi.api_config)

### API Requests

In [ ]:
# setup the time interval

from datetime import timedelta
from ewxpwsdb.time_intervals import previous_fourteen_minute_interval, UTCInterval
interval = UTCInterval.previous_fifteen_minutes()

# this is 1 hour of data, at least 1 hour ago
s = interval.start  - timedelta(minutes = 90)
e = interval.end  - timedelta(minutes=30)

(s,e)


Get the data from an API request, and save that API_response into the database

In [ ]:
# check that the locomos api is working
# this is a special LOCOMOS-only api
if station_type == 'LOCOMOS':
    print(wapi._get_variables())

In [ ]:
# get a response and check it
api_response_records = wapi.get_readings(start_datetime=s, end_datetime=e)
from pprint import pprint
pprint(api_response_records[0].response_text)

In [ ]:
# confirm there is some data in the response

for response_record in api_response_records:
    if wapi.data_present_in_response(response_record):
        print('data found')

import json
from pprint import pprint
response_data = json.loads(api_response_records[0].response_text)
for element in response_data: 
    print(element)

save the api responses from the request in the database, which then assigns and ID number(s)

In [ ]:
session = Session(engine)
for response in api_response_records:
    session.add(response)
    session.commit()

# session is still open

check that the current records that are inside the weatherapi were assigned a database ID

In [ ]:
resp:APIResponse = wapi._get_readings(s, e, interval=60)

### Readings

transform/harmonize the response data into sensor values.  This current requires an open session (opened above) to work as the transform is looking for a database ID on the response record(s).    This should be refactored so this is not the case. 

See the [Collector](../src/ewxpwsdb/collector.py) module for a class that requests, transforms and saves to the db. 

In [ ]:
# note this requires a currently open session to create the foreign key.  
readings = wapi.transform(api_response_records)
readings

In [ ]:
# using the existing session, save the readings to the db
for reading in readings:
    try:
        session.add(reading)
        session.commit()
    except Exception as e:
        print(e)



In [ ]:
print(readings[0].id)

### Reading Insert procedure

what happens when we pull data that we've already inserted?    test our DIY **upsert** method here

In [ ]:

try:
    session.close()
except Exception as e:
    pass

with Session(engine) as session:
    api_response_record = session.get(APIResponse, 1)

x = api_response_record.response_text
x

In [ ]:
print(api_response_record.id)
print(api_response_record.response_text)
readings = wapi.transform([api_response_record])
readings


In [ ]:
# even though re-inserting data with conflicting constraint, should not error (will just insert anyway)
with Session(engine) as session:
    session.add(readings[0])


In [ ]:
stmt = select(Reading)
results = session.exec(stmt)
for r in results.all():
    print(r)

## could we make the above form look more professional?

current it takes two function calls to get a response and transform into reading records. 

Could we do something that looks like this?

```readings = wapi.get_readings(s,e).transform()```

that looks nice, but requires changing the code  to return self from get_readings 

*(which may be possible if the outputs from these functions are saved as state in the class)*

We could also just create a convenience functions to accomplish the same task: 

```readings = wapi.get_and_transform_readings(s,e)  ```


but we also want to save the responses in a database and link the reading records with the response for auditing to discover mistakes, and this does not all for that, 
because the readings don't hvae the response database id to create a link with until they are saved in the DB

The solution is to use the `Collector` class that has the convenience wrappers and combines API with database functions. 

see [`example using collector`](example_using_collector.ipynb) notebook


check that these data can be turned into a Reading object (data + metadata )

save the rows of data from the sensor into the database using a Session

In [ ]:
# readings from transform...
for reading in readings:
    session.add(reading)    
session.commit()
# does it have an id now?
readings[0].id    

In [ ]:

print(readings[0].id )

## Getting data back from the database

the Collector class is great but it has the intenial side of effect of inserting records.   Create new classes that wrap the SQLModels with functionality

this seems redundant but it allows the combination of a database engine and sqlmodels in the top level of the project, and hence the model module does not require database connection. 

- **Station**: additional functionality around the station class and WeatherAPI class which also contains some metadata.   
- **StationReadings**: class that combines stations and readings as readings are almost always used in a station context

In [ ]:
# check we still have a station and engine
print(station)
print(isinstance(station, WeatherStation))
print(f"active engine: {engine}")


In [ ]:

# summarize readings in the database

from ewxpwsdb.station_readings import StationReadings
from ewxpwsdb.station import Station

station_readings = StationReadings(station = station, engine = engine)
print(f"does this station have readings? {station_readings.has_readings()}")
print(f"date of first reading {station_readings.first_reading_date()}")
print("most recent readings in db:")

readings = station_readings.recent_readings()
print(readings)




In [ ]:

# method for pulling readings without the station class

# with Session(engine) as session:
#     stmt = select(Reading, WeatherStation).join(WeatherStation).where(WeatherStation.id  == station.id)
#     # results = session.exec(stmt)
#     reading_records = session.exec(stmt).all()
#     for reading_records in reading_records:
#         reading = reading_records.Reading
#         print(f"{reading.data_datetime}: air temp {reading.atmp}C")

In [ ]:
# get more data
# note that the 'session' must be present for the whole transaction of response and readings for
responses = wapi.get_readings()
session = Session(engine)
for response in responses:
    session.add(response)
    session.commit()

responses[0].id
    
responses[0]

In [ ]:
stmt = select(APIResponse).where(APIResponse.request_id == wapi.current_api_response_records[0].request_id)
result = session.exec(stmt)
some_apiresponse = result.first()
print(some_apiresponse.id)

## Clean up 

If using databases, remove test databases

In [ ]:
# if postgresl (the drop function detects if it's a postgres engine)


from ewxpwsdb.db.database import list_pg_databases, drop_pg_db
from sqlalchemy.orm import close_all_sessions

tmp_db_name = engine.url.database
print(f"attempting to drop db {tmp_db_name}")
close_all_sessions()
engine.dispose()
result = drop_pg_db(tmp_db_name, db_url)
print(result)
print('remaining databases')
print(list_pg_databases(db_url))
